In [ ]:
import os
import pandas as pd
import requests
import gradio as gr
import re

# !pip install pyarrow

In [ ]:
# Load incident‑response dataset
try:
    df = pd.read_parquet("train.parquet")
except ImportError as e:
    raise ImportError(
        "Unable to read Parquet file: please install pyarrow or fastparquet.\n"
        "You can run: pip install pyarrow\n"
    )


# Rename single unnamed column to 'text'
if len(df.columns) == 1:
    df.columns = ['text']
    

# KEY = "xxxxx"

In [18]:
# Clean and parse each row into 'instruction' and 'response' by splitting at '[/INST]'
def split_row(row_text):
    # Remove Parquet artifacts and system tags
    text = row_text
    # Strip leading <s> and SYS wrapper
    text = re.sub(r"^<s>\[INST\] <<SYS>>", "", text)
    text = re.sub(r"<</SYS>>", "", text)
    # Split into user instruction and model response
    parts = text.split('[/INST]')
    if len(parts) >= 2:
        instruction = parts[0].strip()
        response = parts[1].strip()
    else:
        instruction = text.strip()
        response = ''
    return instruction, response

# Apply parsing
parsed = df['text'].apply(split_row)
df['instruction'], df['response'] = zip(*parsed)

In [27]:
df.head(5)

,text,instruction,response
0,<s>[INST] <<SYS>>\nYou are a highly advanced A...,You are a highly advanced AI assistant in our ...,Sure. Here is the information requested.\nTitl...
1,<s>[INST] <<SYS>>\nYou are a highly advanced A...,You are a highly advanced AI assistant in our ...,Sure. Here is the information requested.\nTitl...
2,<s>[INST] <<SYS>>\nYou are a highly advanced A...,You are a highly advanced AI assistant in our ...,Sure. Here is the information requested.\nTitl...
3,<s>[INST] <<SYS>>\nYou are a highly advanced A...,You are a highly advanced AI assistant in our ...,Sure. Here is the information requested.\nTitl...
4,<s>[INST] <<SYS>>\nYou are a highly advanced A...,You are a highly advanced AI assistant in our ...,Sure. Here is the information requested.\nTitl...


In [15]:
MODEL = "llama-3.3-70b-versatile"
API_URL = "https://api.groq.com/openai/v1/chat/completions"

In [ ]:
# LLM call, few-shot learning style

def ask_playbook(api_key, question):
    if not api_key or api_key.strip().lower().startswith("enter"):
        return "Error: Please provide a valid API Key."

    examples = df[['instruction', 'response']].head(5).to_dict(orient="records")
    context = "\n\n".join(
        f"### Example Prompt:\n{ex['instruction']}\n\n### Example Playbook:\n{ex['response']}"
        for ex in examples
    )

    prompt = f"""
You are a cyber-incident playbook generator following NIST SP 800-61 Rev.2.  
Below are sample prompts and generated playbooks:

{context}

Now, generate a detailed, NIST-compliant incident response playbook for this scenario:
"{question}"
"""
    try:
        resp = requests.post(
            "https://api.groq.com/openai/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            },
            json={
                "model": MODEL,
                "messages": [
                    {"role": "system", "content": "You generate NIST-style incident response playbooks."},
                    {"role": "user", "content": prompt}
                ],
                "temperature": 0.2
            },
            timeout=30
        )
    except requests.exceptions.RequestException as e:
        return f"Error: Network or timeout issue: {e}"

    if resp.status_code != 200:
        try:
            err = resp.json()
        except Exception:
            err = resp.text
        return f"Error {resp.status_code}: {err}"

    data = resp.json()
    try:
        return data['choices'][0]['message']['content']
    except (KeyError, IndexError) as e:
        return f"Error parsing response: {e} - {data}"

In [25]:
def main():
    interface = gr.Interface(
        fn=ask_playbook,
        inputs=[
            gr.Textbox(label="API Key", type="password", placeholder="Enter your Groq API Key here"),
            gr.Textbox(label="Incident Scenario", lines=2,
                       placeholder="e.g. Ransomware on Windows Server 2019")
        ],
        outputs=gr.Textbox(label="Generated Playbook"),
        title="Incident Response Playbook Demo",
        description="Enter a scenario and receive a NIST-compliant response playbook."
    )
    interface.launch()

In [ ]:
# KEY = "xxxxx"     Enter your own acount API KEY when prompted

if __name__ == "__main__":
    main()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
